# 📓 01_player_sampling_and_stats.ipynb
**Purpose:** Load player data, inspect it, and prepare for performance modeling.

In [ ]:
import pickle
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from golf_classes import Player, PlayerRoundInfo, Tournament, Round, Team

In [ ]:

import importlib
import golf_classes 

importlib.reload(golf_classes)

Player = golf_classes.Player
PlayerRoundInfo = golf_classes.PlayerRoundInfo
Tournament = golf_classes.Tournament
Round = golf_classes.Round
Team = golf_classes.Team



calcutta_data = {
    "players": {},
    "tournaments": {},
    "teams": []
}

players = calcutta_data["players"]
tournaments = calcutta_data["tournaments"]
teams = calcutta_data["teams"]  # Optional global team structure if used

In [ ]:
def reset_all_data():
    global players, tournaments, teams
    players.clear()
    tournaments.clear()
    teams.clear()
    print("🧹 All data has been reset.")

In [ ]:


golf_data_file = "golf_data.pkl"


def load_data(path=golf_data_file):
    global calcutta_data, players, tournaments, teams
    with open(path, "rb") as f:
        loaded = pickle.load(f)

    calcutta_data["players"] = loaded.get("players", {})
    calcutta_data["tournaments"] = loaded.get("tournaments", {})
    calcutta_data["teams"] = loaded.get("teams", [])

    # Rebind the globals for convenience
    players = calcutta_data["players"]
    tournaments = calcutta_data["tournaments"]
    teams = calcutta_data["teams"]

    print("✅ Data loaded successfully.")
    print(f"players: {len(players)}, teams: {len(teams)}, tournaments: {len(tournaments)}")

In [ ]:
reset_all_data()

load_data() 

In [ ]:
# --- Inspect Example Player ---
example_player = next(iter(players.values() if isinstance(players, dict) else players))

print(f"Example Player: {example_player.name}")
print(f"Total Rounds Recorded: {len(example_player.rounds)}")

# Print some details for the first few rounds
for i, round_info in enumerate(example_player.rounds[:5]):  # Show up to 5 rounds
    print(f" Round {i+1}:")
    print(f"  Tournament: {round_info.tournament_name}")
    print(f"  Round Number: {round_info.round_number}")
    print(f"  Handicap: {round_info.handicap}")
    print(f"  Tee: {round_info.tee}")
    print(f"  Net Score: {round_info.net}")
    print(f"  Total Gross Score: {round_info.total}")

In [ ]:
# --- Statistics ---

# Make sure players is either a list or dictionary
player_list = players.values() if isinstance(players, dict) else players

# Number of players
num_players = len(player_list)

# Number of rounds per player
rounds_per_player = [len(player.rounds) for player in player_list]

# All net scores and valid handicaps across all players and rounds
all_net_scores = []
all_handicaps = []

for player in player_list:
    for round_info in player.rounds:
        all_net_scores.append(round_info.net)
        if pd.notnull(round_info.handicap):
            all_handicaps.append(round_info.handicap)

# Print statistics
print(f"\nStatistics:")
print(f"Total Players: {num_players}")
print(f"Average Rounds per Player: {np.mean(rounds_per_player):.2f}")
print(f"Min Rounds per Player: {np.min(rounds_per_player)}")
print(f"Max Rounds per Player: {np.max(rounds_per_player)}")

valid_net_scores = [s for s in all_net_scores if s is not None and isinstance(s, (int, float))]
if valid_net_scores:
    print(f"\nNet Score Statistics:")
    print(f"  Average Net Score: {np.mean(valid_net_scores):.2f}")
    print(f"  Net Score Range: {np.min(valid_net_scores)} to {np.max(valid_net_scores)}")

valid_handicaps = [h for h in all_handicaps if h is not None and isinstance(h, (int, float))]
if valid_handicaps:
    print(f"\nHandicap Statistics:")
    print(f"  Average Handicap: {np.mean(valid_handicaps):.2f}")
    print(f"  Handicap Range: {np.min(valid_handicaps)} to {np.max(valid_handicaps)}")
else:
    print("\n⚠️ Warning: No valid handicaps found.")


In [ ]:
# --- Plot Net Score Distribution ---
plt.hist(valid_net_scores, bins=30, edgecolor='black')
plt.title('Distribution of Net Scores (All Players)')
plt.xlabel('Net Score')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# --- Plot Handicap Distribution ---
plt.hist(valid_handicaps, bins=range(int(min(all_handicaps)) - 1, int(max(all_handicaps)) + 2), edgecolor='black')
plt.title('Distribution of Handicaps (All Players)')
plt.xlabel('Handicap')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
# --- Optional: Plot rounds per player ---
plt.hist(rounds_per_player, bins=range(0, max(rounds_per_player)+5, 2), edgecolor='black')
plt.title('Distribution of Rounds per Player')
plt.xlabel('Number of Rounds')
plt.ylabel('Number of Players')
plt.grid(True)
plt.show()

In [ ]:
# --- Ready for Sampling Model ---

def sample_score_simple(player):
    return random.choice(player.historical_scores)

def sample_score_weighted(player, decay_factor=0.9):
    scores = player.historical_scores
    n = len(scores)
    weights = [decay_factor ** (n - i - 1) for i in range(n)]
    return random.choices(scores, weights=weights, k=1)[0]

In [ ]:
# --- Test Sampling ---
print("\n🧪 Sampling Test:")
print(f"Simple Sample for {example_player.name}: {sample_score_simple(example_player)}")
print(f"Weighted Sample for {example_player.name}: {sample_score_weighted(example_player)}")

In [ ]:
# --- Save player names (optional) ---
player_names = [p.name for p in (players.values() if isinstance(players, dict) else players)]

In [ ]:
def inspect_player_net_stats(player_name):
    import numpy as np

    player = players.get(player_name)
    if not player:
        print(f"❌ Player '{player_name}' not found.")
        return

    indiv_nets = [
        r.net for r in player.rounds
        if r.tournament_name.lower() == "individual" and r.completed and isinstance(r.net, (int, float))
    ]
    tour_nets = [
        r.net for r in player.rounds
        if r.tournament_name.lower() != "individual" and r.completed and isinstance(r.net, (int, float))
    ]

    print(f"\n📋 Net Score Summary for {player.name}")
    print(f"  ✅ Completed Individual Rounds: {len(indiv_nets)}")
    print(f"  ✅ Completed Tournament Rounds: {len(tour_nets)}")

    if indiv_nets:
        avg_indiv = np.mean(indiv_nets)
        print(f"  🧍 Avg Individual Net: {avg_indiv:.2f}")
    else:
        avg_indiv = None
        print("  🧍 No individual net scores found.")

    if tour_nets:
        avg_tour = np.mean(tour_nets)
        print(f"  🏌️ Avg Tournament Net: {avg_tour:.2f}")
    else:
        avg_tour = None
        print("  🏌️ No tournament net scores found.")

    if avg_indiv is not None and avg_tour is not None:
        print(f"  🔁 Difference (Individual - Tournament): {avg_indiv - avg_tour:.2f}")

In [ ]:
inspect_player_net_stats("David Blacker")

In [ ]:
def compare_net_stats_individual_vs_tournament(top_n=20, min_rounds=2):
    import numpy as np

    player_diffs = []
    all_avg_tour = []
    all_avg_indiv = []

    for player in players.values():
        indiv_nets = [
            r.net for r in player.rounds
            if r.tournament_name.lower() == "individual" and r.completed and isinstance(r.net, (int, float))
        ]
        tour_nets = [
            r.net for r in player.rounds
            if r.tournament_name.lower() != "individual" and r.completed and isinstance(r.net, (int, float))
        ]

        if len(indiv_nets) >= min_rounds and len(tour_nets) >= min_rounds:
            avg_indiv = np.mean(indiv_nets)
            avg_tour = np.mean(tour_nets)
            diff = avg_indiv - avg_tour
            all_avg_tour.append(avg_tour)
            all_avg_indiv.append(avg_indiv)
            player_diffs.append((player.name, avg_tour, avg_indiv, diff, len(tour_nets), len(indiv_nets)))

    # Sort by absolute difference, descending
    player_diffs.sort(key=lambda x: x[3], reverse=True)

    # Print
    header = "{:<5} {:<25} {:>15} {:>15} {:>12} {:>10} {:>10}".format(
        "Rank", "Player", "Avg Tournament", "Avg Individual", "Difference", "Tour Rds", "Indiv Rds"
    )
    print(f"\n📊 Top {top_n} Players with Largest Net Score Differences (Min {min_rounds} Rounds Each):\n")
    print(header)
    print("-" * len(header))

    for i, (name, avg_tour, avg_indiv, diff, tour_ct, indiv_ct) in enumerate(player_diffs[:top_n], start=1):
        print("{:<5} {:<25} {:>15.2f} {:>15.2f} {:>12.2f} {:>10} {:>10}".format(
            i, name, avg_tour, avg_indiv, diff, tour_ct, indiv_ct
        ))

In [ ]:
# --- Example Usage ---
print(f"players: {len(players)}, teams: {len(teams)}, tournaments: {len(tournaments)}")

total_players = len(players)
compare_net_stats_individual_vs_tournament(top_n=total_players, min_rounds=3)

In [ ]:
def calculate_and_print_player_net_stats(top_n=20, min_rounds=5):
    import numpy as np

    player_stats = []
    player_list = players.values() if isinstance(players, dict) else players

    for player in player_list:
        net_scores = [
            r.net for r in player.rounds
            if r.net is not None and isinstance(r.net, (int, float)) and getattr(r, "completed", False)
        ]
        if len(net_scores) >= min_rounds:
            avg_net = np.mean(net_scores)
            std_net = np.std(net_scores)
            player_stats.append((player.name, avg_net, std_net, len(net_scores)))

    player_stats.sort(key=lambda x: x[1])

    header = "{:<5} {:<25} {:>15} {:>12} {:>10}".format("Rank", "Player Name", "Avg Net Score", "Std Dev", "Rounds")
    print(f"\n🏌️ Top {top_n} Players by Average Net Score (Minimum {min_rounds} Completed Rounds):\n")
    print(header)
    print("-" * len(header))

    for idx, (name, avg_score, std_dev, num_rounds) in enumerate(player_stats[:top_n], start=1):
        row = "{:<5} {:<25} {:>15.2f} {:>12.2f} {:>10}".format(idx, name, avg_score, std_dev, num_rounds)
        print(row)

In [ ]:

# --- Example Usage ---
print(f"players: {len(players)}, teams: {len(teams)}, tournaments: {len(tournaments)}")
calculate_and_print_player_net_stats(top_n=50, min_rounds=4)